In [448]:
pip install tensorflow

zsh:1: no such file or directory: /usr/local/bin/python3.10
Note: you may need to restart the kernel to use updated packages.


# Risk Assesement

In [1]:
%matplotlib inline

In [554]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.seasonal import STL
from scipy.stats import ks_2samp
from Portfolio import selected_tickers

import math
import yfinance as yf
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import datetime as time

In [3]:
stock_data_june = pd.read_csv("stock_market_june2025.csv")
stock_data_53 = pd.read_csv("stock_data_july_2025.csv")

In [253]:
pd.set_option('display.max_rows', 100) 
stock_data_june['Ticker'].value_counts().gt(2).sum()

0

In [5]:
# Checking tickers that appear more than once
stock_data_53['Ticker'].value_counts().gt(1).sum()
# Checking tickers that appear more than twice
stock_data_53['Ticker'].value_counts().gt(2).sum()

# Filtering
valid_tickers = stock_data_53['Ticker'].value_counts()
valid_tickers = valid_tickers[valid_tickers > 2].index
sorted_stock_data = stock_data_53[stock_data_53['Ticker'].isin(valid_tickers)]

In [6]:
len(sorted_stock_data)

4346

In [7]:
# Sorting the df
sorted_stock_data["Date"] = pd.to_datetime(sorted_stock_data["Date"], dayfirst=True, format="mixed")
sorted_stock_data = sorted_stock_data.sort_values(['Ticker', 'Date'])


### Percentage Change

In [8]:
# Calculating percentage change
sorted_stock_data['Return'] = sorted_stock_data.groupby('Ticker')['Close Price'].pct_change()
# sorted_stock_data

### Volatility

In [52]:
# Volatility
volatility = sorted_stock_data.groupby('Ticker')['Return'].std()
volatility.dropna(inplace=True)
volatility.to_csv('volatility-by-ticker.csv')
# volatility

### Value At Risk

In [51]:
# Value at Risk
Var_95_by_ticker = sorted_stock_data.groupby('Ticker')['Return'].apply(
    lambda x: np.percentile(x.dropna(), 5)
)
Var_95_by_ticker.to_csv('VaR-by-ticker.csv')
Var_95_by_ticker

### Conditional VaR

In [11]:
# Conditional VaR
CVaR_95_by_Ticker = sorted_stock_data.groupby('Ticker')['Return'].apply(
    lambda x: x[x <= np.percentile(x.dropna(), 5)].mean()
)
CVaR_95_by_Ticker.to_csv('CVaR-by-Ticker.csv')

### Drawdown

In [12]:
# Cumulative Return Calculation
sorted_stock_data['CumulativeReturn'] = (1 + sorted_stock_data['Return']).groupby(sorted_stock_data['Ticker']).cumprod()


In [13]:
# Running Maximum Calcualtion
sorted_stock_data['RunningMax'] = sorted_stock_data.groupby('Ticker')['CumulativeReturn'].cummax()

In [14]:
# Drawdown Calculation
sorted_stock_data['Drawdown'] = sorted_stock_data['CumulativeReturn'] / sorted_stock_data['RunningMax'] - 1
max_drawdown = sorted_stock_data.groupby('Ticker')['Drawdown'].min()
sorted_stock_data.to_csv('data-with-drawdown.csv')

In [15]:
stock_data = stock_data_june

In [16]:
stock_data_53['Date']= pd.to_datetime(stock_data_53['Date'])
stock_data_june['Date']= pd.to_datetime(stock_data_june['Date'], format='%d-%m-%Y')

filtered = stock_data_53[stock_data_53['Date'] <= '2025-06-30']
stock_data_june.reset_index(drop=True).equals(filtered.reset_index(drop=True))

False

In [17]:
stock_data_june['Ticker'].value_counts()

Ticker
OUF    2
XVD    2
CLL    2
GFF    2
TXP    2
      ..
JKX    1
OUV    1
KOM    1
LJX    1
UAA    1
Name: count, Length: 1691, dtype: int64

In [50]:
filtered['Ticker'].value_counts()

# Time Series Analysis

# Scrapping

In [229]:
dataset_tickers = stock_data_53['Ticker'].unique().tolist()
dataset_tickers = [t.replace('.', '-') for t in dataset_tickers]
ten_year_data = yf.download(dataset_tickers, start='2014-06-01', end='2025-06-01', threads=False)[["Open","High","Low","Close", "Volume"]]


[*********************100%***********************]  82 of 82 completed


In [308]:
# Removing UBER because it started trading publiclly in 2019, and some values for PYPL are also missing
ten_year_data.drop(['UBER', 'PYPL'], axis=1, level=1, inplace=True)

In [293]:
pd.reset_option('display.max_rows')

In [292]:
mask = temp.isna()
stacked = mask.stack(level=list(range(temp.columns.nlevels)), dropna=False)
nan_index = stacked[stacked].index.tolist()
cols = ['row'] + [f'col_level_{i}' for i in range(temp.columns.nlevels)]
nan_df = pd.DataFrame(nan_index, columns=cols)

#### There are missing values. These values are refilled by forward fill

In [309]:
missing_days = ten_year_data.index.to_series().diff().dt.days.value_counts()
missing_days

Date
1.0    2166
3.0     497
4.0      76
2.0      27
Name: count, dtype: int64

In [310]:
(ten_year_data.isnull() == True).any().any()

False

In [311]:
ten_year_data.shape

(2767, 400)

In [312]:
date_without_missing = pd.date_range(start=ten_year_data.index.min(), end=ten_year_data.index.max())
ten_year_data_fill = ten_year_data.reindex(date_without_missing).ffill()

In [313]:
ten_year_data_fill.shape

(4016, 400)

In [314]:
(ten_year_data_fill.isnull() == True).any().any()

False

In [315]:
ten_year_data_fill['Close']

Ticker,AAPL,ABBV,ABT,ADBE,ADP,AMD,AMGN,AMT,AMZN,AVGO,...,TXN,UNH,UNP,UPS,V,VRTX,VZ,WMT,XOM,ZTS
2014-06-02,19.744541,34.322792,32.082146,64.639999,54.251503,3.970000,84.674370,68.327972,15.442000,5.369713,...,34.601234,66.636658,77.222740,70.447121,49.329594,72.629997,28.186136,20.540117,62.015350,28.361816
2014-06-03,20.023756,34.462227,32.098282,64.089996,53.793621,3.940000,85.161552,68.583046,15.359500,5.355471,...,34.601234,66.980446,76.532623,70.007156,48.821339,72.959999,27.758133,20.526731,62.294598,28.664511
2014-06-04,20.252405,34.595333,32.041847,64.169998,53.875641,4.040000,86.637642,68.807228,15.339000,5.367465,...,34.498142,67.508690,76.703209,69.695824,48.846752,72.639999,27.679289,20.639118,62.077431,28.774586
2014-06-05,20.331875,35.051701,32.364288,65.470001,53.786781,4.080000,85.612358,69.363815,16.178499,5.386954,...,34.903141,66.913376,77.676384,70.135765,49.029259,73.059998,27.752506,20.689960,62.393883,28.994730
2014-06-06,20.275961,34.924938,32.283691,66.910004,54.408703,4.060000,85.670540,69.897186,16.483500,5.368964,...,34.976765,67.022369,78.265747,70.115463,49.209469,73.489998,27.831345,20.660536,63.045429,29.113977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-26,195.048645,181.690720,130.713272,407.690002,317.842712,110.309998,269.473938,210.205597,200.990005,228.182922,...,175.034866,293.486603,219.402191,93.602295,352.919006,436.000000,42.630703,96.115509,102.080101,162.038177
2025-05-27,199.983047,184.129669,132.345932,413.100006,322.415985,114.559998,277.201263,212.031235,206.020004,235.096649,...,181.925064,292.920593,221.614899,95.741890,358.668884,446.000000,42.807838,97.352623,102.565582,165.705902
2025-05-28,200.192795,181.522171,131.430054,412.230011,320.861908,112.860001,276.159698,210.800919,204.720001,238.867767,...,182.828522,296.008667,219.471329,94.946899,359.098145,445.100006,42.443726,97.013412,101.168587,164.848770
2025-05-29,199.723328,184.030518,132.256348,413.359985,320.842072,113.029999,281.258362,212.041168,205.699997,241.401810,...,183.662506,296.068268,219.797318,96.281693,361.763428,447.089996,42.640545,96.873741,101.743240,166.582977


## Returns

In [316]:
ten_year_returns = ten_year_data_fill['Close'].pct_change()

## Features

### Moving Averages

In [317]:
mo_a_5 = ten_year_returns.rolling(5).mean()
mo_a_5.columns = pd.MultiIndex.from_product([['MA_5'], mo_a_5.columns])
mo_a_10 = ten_year_returns.rolling(10).mean()
mo_a_10.columns = pd.MultiIndex.from_product([['MA_10'], mo_a_10.columns])
mo_a_20 = ten_year_returns.rolling(20).mean()
mo_a_20.columns = pd.MultiIndex.from_product([['MA_20'], mo_a_20.columns])
mo_a_50 = ten_year_returns.rolling(50).mean()
mo_a_50.columns = pd.MultiIndex.from_product([['MA_50'], mo_a_50.columns])
mo_a_100 = ten_year_returns.rolling(100).mean()
mo_a_100.columns = pd.MultiIndex.from_product([['MA_100'], mo_a_100.columns])
mo_a_200 = ten_year_returns.rolling(200).mean()
mo_a_200.columns = pd.MultiIndex.from_product([['MA_200'], mo_a_200.columns])

In [318]:
moving_averages = pd.concat([
    mo_a_5,
    mo_a_10,
    mo_a_20,
    mo_a_50,
    mo_a_100,
    mo_a_200
], axis=1)

#### Scaling

In [319]:
# using standardization scaling on my data because i want to calculate returns
scaled_array = StandardScaler().fit_transform(moving_averages)
scaled_moving_averages = pd.DataFrame(
    scaled_array,
    index=moving_averages.index,
    columns=moving_averages.columns
)

### Volatility

In [320]:
vol_21 = ten_year_returns.rolling(21).std()
vol_63 = ten_year_returns.rolling(63).std()
vol_252 = ten_year_returns.rolling(252).std()

#### Scaling

In [321]:
scaled_vol_21 = pd.DataFrame(
    StandardScaler().fit_transform(vol_21),
    index= vol_21.index,
    columns= vol_21.columns
)
scaled_vol_63 = pd.DataFrame(
    StandardScaler().fit_transform(vol_63),
    index= vol_63.index,
    columns= vol_63.columns
)
scaled_vol_252 = pd.DataFrame(
    StandardScaler().fit_transform(vol_252),
    index= vol_252.index,
    columns= vol_252.columns
)

### Market Index

In [322]:
market_index = yf.download('^GSPC', start='2014-06-01', end='2025-06-01')

[*********************100%***********************]  1 of 1 completed


In [344]:
market_index.index.to_series().diff().value_counts()

1 days    4015
Name: count, dtype: int64

In [327]:
missing = pd.date_range(market_index.index.min(), market_index.index.max())
market_index = market_index.reindex(missing).ffill()

In [328]:
market_index_return = market_index['Close'].pct_change().dropna()

In [329]:
market_index_return.shape

(4015, 1)

#### Scaling

In [330]:
sc_market_index_returns = pd.DataFrame(
    StandardScaler().fit_transform(market_index_return),
    index=market_index_return.index,
    columns=market_index_return.columns
)

### RSI

In [331]:
delta = ten_year_data_fill['Close'].diff()

gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)

avg_gain = gain.ewm(alpha=1/14, min_periods=14, adjust=False).mean()
avg_loss = loss.ewm(alpha=1/14, min_periods=14, adjust=False).mean()

rs = avg_gain/avg_loss

rsi = 100 - (100/(1 + rs))

#### Scaling

In [332]:
scaled_rsi = pd.DataFrame(
    StandardScaler().fit_transform(rsi),
    index=rsi.index,columns=rsi.columns
)

### MACD

In [333]:
fast_emw = ten_year_data_fill['Close'].ewm(alpha=1/12,adjust=False).mean()
slow_emw = ten_year_data_fill['Close'].ewm(alpha=1/26, adjust=False).mean()
macd_line = fast_emw - slow_emw

signal_line = macd_line.ewm(alpha=1/9, adjust=False).mean()

macd_histogram = macd_line - signal_line

#### Scaling

In [334]:
sc_macd_line = pd.DataFrame(
    StandardScaler().fit_transform(macd_line),
    index=macd_line.index,columns=macd_line.columns
)
sc_signal_line = pd.DataFrame(
    StandardScaler().fit_transform(signal_line),
    index=signal_line.index,columns=signal_line.columns
)
sc_macd_histogram = pd.DataFrame(
    StandardScaler().fit_transform(macd_histogram),
    index=macd_histogram.index,columns=macd_histogram.columns
)

### Bollinger Bands

In [339]:
sma_20 = ten_year_data_fill['Close'].rolling(20).mean()
roll_std = ten_year_data_fill['Close'].rolling(20).std()

upper_band = sma_20 + (2 * roll_std)
lower_band = sma_20 - (2 * roll_std)

sma_20.columns = pd.MultiIndex.from_product([['SMA-20'], sma_20.columns])
upper_band.columns = pd.MultiIndex.from_product([['UpperBand'], upper_band.columns])
lower_band.columns = pd.MultiIndex.from_product([['LowerBand'], lower_band.columns])

bollinger_bands = pd.concat([
    sma_20,
    upper_band,
    lower_band
], axis=1)


#### Scaling

In [340]:
sc_bollinger_bands = pd.DataFrame(
    StandardScaler().fit_transform(bollinger_bands),
    index=bollinger_bands.index,columns=bollinger_bands.columns
)

### On-Balance Volume

In [499]:
OBV = pd.DataFrame(0, index=ten_year_data_fill['Close'].index, columns=ten_year_data_fill['Close'].columns)
for j in range(0, len(ten_year_data_fill['Close'].columns)):
    close_col = ten_year_data_fill['Close'].iloc[:,j].values
    volume_col = ten_year_data_fill['Volume'].iloc[:,j].values
    obv_col = np.zeros(len(close_col))
    for i in range(1, len(close_col)):
        if close_col[i] > close_col[i-1]:
            obv_col[i] = obv_col[i-1] + volume_col[i]
        elif close_col[i] < close_col[i-1]:
            obv_col[i] = obv_col[i-1] - volume_col[i]
        else:
            obv_col[i] = obv_col[i-1]
    OBV.iloc[:,j]=obv_col 

### Scaling

In [343]:
scaled_OBV = pd.DataFrame(
    StandardScaler().fit_transform(OBV),
    index=OBV.index,columns=OBV.columns
)

In [375]:
sc_market_index_returns.columns.nlevels > 1

False

## Training LSTM for AAPL

### Inputs For LSTM

In [539]:
aapl_features = pd.concat([
    scaled_moving_averages.xs('AAPL', level=1, axis=1),
    scaled_vol_21['AAPL'].rename('VOL_21'),
    scaled_vol_63['AAPL'].rename('VOL_63'),
    scaled_vol_252['AAPL'].rename('VOL_252'),
    sc_market_index_returns.rename(columns={'^GSPC':'MARKET_INDEX'}),
    scaled_rsi['AAPL'].rename('RSI'),
    sc_macd_line['AAPL'].rename('MACD_LINE'),
    sc_signal_line['AAPL'].rename('SIGNAL_LINE'),
    sc_macd_histogram['AAPL'].rename('MACD_HIST'),
    sc_bollinger_bands.xs('AAPL', level=1, axis=1),
    scaled_OBV['AAPL'].rename('OBV')
], axis=1)

In [545]:
aapl_features.dropna(how='any', inplace=True)

### Sliding Window Generation

In [559]:
timesteps = 60
horizon = 21

aapl_features_array = aapl_features.values
aapl_returns = ten_year_returns['AAPL'].dropna().values

X, y = [], []
for i in range(len(aapl_features_array) - timesteps - horizon):
    X.append(aapl_features_array[i:i+timesteps])
    y.append(aapl_returns[i+timesteps:i+timesteps+horizon])

X = np.array(X)
y = np.array(y)

In [561]:
# splititng the data
split_point = int(0.8 * len(X))
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

In [571]:
np.all(X_train.inde[:-1] <= X_train[1:])

False

In [562]:
model = Sequential()
model.add(LSTM(64, return_sequences=False, input_shape=(timesteps, X.shape[2])))
model.add(Dropout(0.3))
model.add(Dense(horizon))
model.compile(optimizer='adam', loss='mse')

In [563]:
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.1
)

Epoch 1/30
83/83 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - loss: 0.0256 - val_loss: 0.0045
Epoch 2/30
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0061 - val_loss: 0.0016
Epoch 3/30
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0030 - val_loss: 9.4992e-04
Epoch 4/30
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0019 - val_loss: 7.1469e-04
Epoch 5/30
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 0.0013 - val_loss: 6.0002e-04
Epoch 6/30
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 0.0010 - val_loss: 4.9164e-04
Epoch 7/30
83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 8.2141e-04 - val_loss: 4.5014e-04
Epoch 8/30
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 6.8820e-04 - val_loss: 4.2911e-04
Epoch 9/30
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 5.8876e-04 - val_loss: 4.0415e-04
Epoch 10/30
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step - loss: 5.3078e-04 - val_loss: 3.6771e-04
Epoch 11/30
83/83 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - loss: 4.7289e-04 - val_loss: 3.7089e-04
Epoch 1

In [564]:
loss = model.evaluate(X_test, y_test)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 2.6780e-04


In [565]:
rmse = math.sqrt(loss)
rmse

0.016364494616593585

In [566]:
aapl_returns.std()

0.015056433538944427

In [473]:
y_pred = model.predict(X_test)

25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step


In [482]:
y_pred[786]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan], dtype=float32)

## Classical Decomposition 

In [41]:
decomposition = {}
for ticker in three_year_data_return.columns:
    series = three_year_data_return[ticker]
    decomposition[ticker] = seasonal_decompose(series, model='additive', period=21)

NameError: name 'three_year_data_return' is not defined

In [ ]:
trend = decomposition["VZ"].trend
seasonsal = decomposition["VZ"].seasonal
residual = decomposition["VZ"].resid

In [ ]:
plt.figure(figsize=(35,20))
plt.subplot(411)
plt.plot(three_year_data_return.index, trend, label = "Trend", color= "blue")
plt.legend(loc="upper left")
plt.subplot(412)
plt.plot(three_year_data_return.index, seasonsal, label = "Seasonality", color= "green")
plt.legend(loc="upper left")
plt.subplot(413)
plt.plot(three_year_data_return.index, residual, label = "Residual", color= "orange")
plt.legend(loc="upper left")
plt.tight_layout()
plt.show()

### STL Decomposition

In [ ]:
stl_decomp = STL(daily_avg["Close Price"], period=7).fit()

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(411)
plt.plot(daily_avg["Date"] ,stl_decomp.observed, label="Original", color="orange")
plt.legend(loc="upper left")
plt.subplot(412)
plt.plot(daily_avg["Date"] ,stl_decomp.trend, label="Trend", color="grey")
plt.legend(loc="upper left")
plt.subplot(413)
plt.plot(daily_avg["Date"], stl_decomp.seasonal, label="Seasonal", color="grey")
plt.legend(loc="upper left")
plt.subplot(414)
plt.plot(daily_avg["Date"], stl_decomp.resid, label="Residual", color="black")
plt.legend(loc="upper left")
plt.tight_layout()
plt.show()

### Checking Stationarity
#### Performing Adf Test

In [ ]:
adf_test = adfuller(daily_avg["Close Price"])
print(f"ADF Statistics: {round(adf_test[0], 3)}")
print(f"p-value: {round(adf_test[1], 3)}")
print("Critical Values")
for key, value in adf_test[4].items():
    print(f"{key}: {round(value, 3)}")


#### Performing KPSS Test

In [ ]:
kpss_test = kpss(daily_avg["Close Price"], regression="ct")
print(f"KPSS Statistics: {round(kpss_test[0], 2)}")
print(f"p-value: {kpss_test[1]}")
for key, value in kpss_test[3].items():
    print(f"{key}: {value}")

#### Performing KS Test to check strict stationarity

In [ ]:
def ks_test_stationarity(series):
    split = len(series) //2
    first_half = series[:split]
    second_half = series[split:]
    stat, p_value  = ks_2samp(first_half, second_half)
    return stat, p_value

ks_stat, ks_p_value = ks_test_stationarity(daily_avg["Close Price"])
print(ks_stat, ks_p_value)

#### Conclusion: 
#### There is a conflict between the adf and kpss tests because the provided data is of 21 days which is not suitable for either of the tests to provide accurate results. Thereby, stationarity of data remains ambiguous